# Init

In [9]:
import pyspark
import os
import sys

from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [10]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [11]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'
sc = SparkSession.builder.appName("lab2").master("local[1]").config("spark.sql.parquet.writeLegacyFormat", "true").getOrCreate()
sc

# Загрузка данных из файла

In [12]:
# Загрузка данных о языках программирования
programming_languages_df = sc.read.csv("data/programming-languages.csv", header=True, inferSchema=True)
programming_languages_list = [row.name for row in programming_languages_df.collect()]

programming_languages_list[:10]

['A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET',
 'ABSYS']

In [13]:
# Загрузка данных из posts_sample.xml с использованием пакета spark-xml
posts_df = sc.read \
    .format("xml") \
    .option("rowTag", "row") \
    .load("data/posts_sample.xml")

# Фильтрация постов типа "вопрос" с указанным периодом времени и извлечение тегов
posts_filtered_df = posts_df \
    .filter("_PostTypeId = 1") \
    .withColumn("Year", col("_CreationDate").substr(1, 4).cast("int")) \
    .filter((col("Year") >= 2010) & (col("Year") <= 2020)) \
    .select("_Tags", "Year")

In [14]:
# Предварительная обработка данных из XML
posts_df = sc.read.format("xml").options(rowTag="row").load("data/posts_sample.xml")
posts_rdd = posts_df.rdd

# Составление отчета

In [15]:
# Функция для определения языка программирования по тегам
def detect_language(post):
    tags_lower = post._Tags.lower()
    detected_language = next((lang for lang in programming_languages_list if f"<{lang.lower()}>" in tags_lower), None)
    return Row(_Id=post._Id, Programming_Language=detected_language) if detected_language is not None else None


# Функция для проверки, что пост был опубликован в определённом году
def is_post_in_year(post, year):
    post_year = post._CreationDate.year
    return post_year == year


# Функция для обработки постов за определённый год
def process_posts_for_year(year, posts_rdd):
    year_results = posts_rdd \
        .filter(lambda post: post._Tags is not None and is_post_in_year(post, year)) \
        .map(detect_language) \
        .filter(lambda result: result is not None) \
        .map(lambda post: (post.Programming_Language, 1)) \
        .reduceByKey(lambda a, b: a + b) \
        .sortBy(lambda x: x[1], ascending=False) \
        .take(10)

    df = sc.createDataFrame(year_results, ["Programming_Language", f"Number_of_Mentions_{year}"])
    return df

# Вывод результатов

In [16]:
# Выполнение обработки по годам и вывод результатов
report = {}
for year in range(2010, 2020):
    report[year] = process_posts_for_year(year, posts_rdd)
    print(f"Top 10 programming languages for {year}:")
    report[year].show()

Top 10 programming languages for 2010:
+--------------------+-----------------------+
|Programming_Language|Number_of_Mentions_2010|
+--------------------+-----------------------+
|                Java|                     52|
|          JavaScript|                     44|
|                 PHP|                     42|
|              Python|                     25|
|         Objective-C|                     23|
|                   C|                     20|
|                Ruby|                     11|
|              Delphi|                      7|
|                   R|                      3|
|                Bash|                      3|
+--------------------+-----------------------+
Top 10 programming languages for 2011:
+--------------------+-----------------------+
|Programming_Language|Number_of_Mentions_2011|
+--------------------+-----------------------+
|                 PHP|                     97|
|                Java|                     92|
|          JavaScript|       